In this project we will summarize and compute the principal physical quantities in a synchrotron emission. We will use atropy units module for this purpose and we will base ourselves on NRAO book chapter 5 (https://www.cv.nrao.edu/~sransom/web/Ch5.html) and ThunderBooks basics course (https://github.com/robfender/ThunderBooks/blob/master/Basics.ipynb). A deeper explanation is found on NRAO book chapter 5 (https://www.cv.nrao.edu/~sransom/web/Ch5.html).

## 1. Introducción

In [95]:
import numpy as np
import matplotlib.pyplot as plt

#Packages to make an interactive plot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget

#Packages for using tables and units in astropy
from astropy.table import QTable
import astropy.units as u
from astropy.io import ascii

#Astropy module to handle units
from astropy import units as u
from astropy.units import Quantity
from astropy import constants as const

#Importing the constant that will be needed
from astropy.constants import c, k_B,m_e,e

Now we define a set of functions that will be used along the project, first one will give the size of an expanding object

In [102]:
def sizefromt(t,v):
    """
    sizefromt gives the radius (meter) and volume (meter^3) that a object starting as a point and expanding a time t
    with constant speed v will have. If units are desired at the output, they must be inserted in the parameters with unit
    module

    Parameters
    :t: Time of the evolution
    :v: Speed at which the object is expanding

    Output
    :r: Radius of the object at t
    :V: Volume of the object at t
    """
    #Checking that input has units equivalent to the right ones
    t=t.to(u.s)
    v=v.to(u.m/u.s)
    
    #Applying simple trigonometrical expressions
    r=t*v
    V=((4.0/3.0)*np.pi*(r**3.0)) 

    return (r,V)

We will be using solid angles in different expression so it is convenient to make a function on order to get it from the source size and distance.

In [97]:
def solidangle(r,d):
    """
    solidangle gives the solid angle of a source at distance d and size r in the small angle approximation.If units are 
    desired at the output, they must be inserted in the parameters with unit module

    Parameters
    :r: size of the source
    :d: distance from Earth to source

    Outputs
    :omega: solid angle of the source
    """
    #Applying the expresion for the solid angle in the small angle approximation
    omega= np.pi * (r**2)/(d**2)
    return omega

In [98]:
def LogLogPlot(x, y, xlabel, ylabel):
    """
    LogLogplot plots a graphic of a given set of points (x,y) with specific y and x labels

    Parameters
    :x: x values of the set of points
    :y: y values of the set of points
    :xlabel: Label for the x axis
    :ylabel: label for the y axis
    """
    #Plotting the points
    plt.plot(x,y)
    #Making the labels
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

## 2. Basic Calculus.

### 2.1 Frequencies, energy and gamma relation.

A relativistic electron on a magnetic frield radiates energy (synchrotron) peaked at a frequency $\nu_c$ wich can be related to the energy radiated by a non relativistic electron a a magnetic field (cyclotron) peaked at a frequency $\nu_g$ with the expression
$$
    \nu_c=\frac{\gamma^2eB}{2\pi m_e c}.
$$

In the practice, when we make an observation, we measure the frequancy at wich we see the peak, and with that we want to infer the characteristics of the electron and the medium. The peak in frequency observed will be related with electrons with a $\gamma$ given by the following expression
$$
    \gamma= \sqrt{\frac{2\pi m_e \nu_c}{eB}}. 
$$
(EXPRESION NO CONCUERDA 5.81 NRAO CON THUNDERBOOK Y LA DE NRAO NO TIENE SENTIDO UNIDADES)

A simple code can be done in order to get this $\gamma$ factor in function of the observed peak frequency.

In [99]:
def gamma(v, B):
    """
    gamma receives the frequency at wich we are observing the peak of emission and the magnetic field associated with the 
    synchrotron emission and returns the gamma factor associated with the electrons producing the radiation

    Parameters
    :v: frequency at with we observe the peak
    :B: modulus of the amgnetic field at the synchrotron region

    Outputs
    :gamma: gamma factor asociated with the relativistic electrons
    """
    #Converting parameters to SI so it is also checked that they are in units equivalent to Hz and Tesla
    v= v.to(u.Hz)
    B = B.to(u.Tesla)
    
    #Computing the gamma using the constant with respective units
    gamma= (2*np.pi*const.m_e*v/(const.e.si*B))**0.5

    #Output the value with reduced units
    return gamma

In [100]:
def egammaenergy(gamma):
    """
    egammaenergy gives the energy (in MeV) of an electron given its gamma factor

    Parameters
    :gamma: gamma factor of the electron

    Output
    :energy: Relativistic energy of the electron in MeV
    """

    #Computing and output the energy with units
    energy= const.m_e*gamma*const.c**2
    return energy.to(u.MeV)

### 2.2 Brightness temperature.

Brightness temperature of an object is defined as the temperature that a black body with the same size will need in order to produce the same flux at that frequency. It is a very usefull maximum, synchrotron maximun emission is around $10^{12} K$ so temepratures above this are either due bad assumption, doppler boost or coherent emission. Brightness temperature can be written as
$$
    T_B= \frac{S_\nu c^2}{2 K_B \Omega \nu^2}
$$

In [101]:
def TB(S, R, d, v):
    """
    TB gives the brightness temperature for a given flux, radius and frequency. It assumes small angle approximation and 
    circular shape of the source

    Parameters
    :S: flux of the source
    :R: radius of the source
    :d: distance from earth to source 
    :v: frequency at wich we want to calculate the brightness temperature

    Output
    :TB: brightness temperature 
    """

    #Makign sure every input is on a unit equivalent to a valid one
    S=S.to(u.Jy)
    R= R.to(u.m)
    d= d.to(u.m)
    v= v.to(u.Hz)

    #Computing the brightness temperature
    TB= S*const.c**2/(2*const.k_B*solidangle(R,d)*v**2)
    return TB.to(u.K)

When we do not have a resolved source we can constrain the size of the source by variability timescale and get a lower limit for the brightness temperature.

In [107]:
def TBfromt(S, t, d, v):
    """
    TBfromt gives a lower limit for brightness temperature for a given flux, frequency and variability timescale. 
    It assumes small angle approximation and circular shape of the source

    Parameters
    :S: flux of the source
    :t: variability timescale
    :d: distance from earth to source 
    :v: frequency at wich we want to calculate the brightness temperature

    Output
    :TB: brightness temperature lower limit
    """

    #Making sure every input is on a unit equivalent to a valid one
    S= S.to(u.Jy)
    t= t.to(u.s)
    d= d.to(u.m)
    v= v.to(u.Hz)

    #Getting the radius from variability timescale
    R=sizefromt(t,const.c)[0]
    
    #Computing the brightness temperature
    TB= S*const.c**2/(2*const.k_B*solidangle(R,d)*v**2)
    return TB.to(u.K)

## 3. Minimun Energy.

It can be shown (NRAO Chapter 5) that a synchrotron source energies stored in electrons and magentic field are proportional to the magnetic field as
$$
    U_e \propto B^{-3/2} ~~~~~~ U_B \propto B^2.
$$

We can assume that the total energy is only given by cosmic rays and magnetic fields, i.e, $U=(1+\beta)U_e+U_B$, where $\beta$ is the ion-electron energy ratio. Under this asumption, the minimun of the energy in function of B can be found and relates the energies as
$$
    (1+\beta)U_e= \frac{3}{4} E_e.
$$
since they both are so symillar this is the so called equipartition principle.

### 3.1 simple model for showing equipartition.

Longair made a simplified formula for medium with spectral index $\alpha=0.75$ with 
$$
    E_{min,SI} \approx 3\cdot 10^6 \eta^{4 / 7} V^{3 / 7} \nu^{2 / 7} L^{4 / 7}~~J,
$$
where V is the emitting volume, $\nu$ is the observed frequency, $L$ is the specific luminosity and every magnitude is on the international system. The magnetic field at this minimun energy is given by
$$
    B_{\operatorname{minE}, \mathrm{SI}}=1.8\left(\frac{\eta L_\nu}{V}\right)^{2 / 7} \nu^{1 / 7}~~T,
$$
where $\eta=1+\beta$.

In [128]:
def Lum(S,d):
    """
    Lum calculates the luminosity of an object with a given observed flux density and it's distance from Earth

    Parameters
    :S: object observed flux density
    :d: object distance from Earth

    Output
    :Lum: luminosity of the object
    """

    #Checking units
    S= S.to(u.Jy)
    d= d.to(u.m)

    #Computing the luminosity
    L= 4*np.pi*d**2*S
    return L

In [129]:
def LongairMinE(eta, V, v, L):
    """
    LongairMinE gives the minimun energy given a electron-proton energy ratio, a volume of emission, frequency of the 
    observed radiation and luminosity observed.

    Parameters
    :eta: fraction of energy carried by electron and ions
    :V: total volume of emission
    :v: frequency of observed radiation
    :L: luminosity of the object

    Output
    :Emin: minimun energy of the system
    """

    #Check that every input is on the right magnitudes
    eta= eta*u.dimensionless_unscaled
    V= V.to(u.m**3)
    v= v.to(u.Hz)
    L= L.to(u.W/u.Hz)

    #Computing the minimun energy and putting it in J as the expression indicates
    Emin= (3*10**6*eta**(4/7)*V**(3/7)*v**(2/7)*L**(4/7))
    return Emin.value*u.J

In [ ]:
def BminE(eta, L, V, v):
    """
    BminE gives the value of the magnetic field at the minumun energy

    Parameters
    :eta: fraction of energy carried by electron and ions
    :V: total volume of emission
    :v: frequency of observed radiation
    :L: luminosity of the object
    
    Outputs
    :BminE: value of the magnetic field at the energy minimun
    """
    #Checking units
    eta= eta*u.dimensionless_unscaled
    V= V.to(u.m**3)
    v= v.to(u.Hz)
    L= L.to(u.W/u.Hz)
    
    BminE= 1.8*(eta*L/V)**(2/7)*v**(1/7)
    return BminE.value*u.Tesla

Let's plot the energies and see the equipartition principle.

Let's plot the 

In [ ]:

x=np.linspace(Beq/100,Beq*100,10000)